In [3]:
from datasets import load_dataset

dataset_name = "dim/nfs_pix2pix_1920_1080_v5"
# dataset_name = "dim/nfs_pix2pix_1920_1080_v5_upscale_2x_raw"
# dataset_name = "dim/nfs_pix2pix_1920_1080_v6"
# dataset_name = "dim/render_nfs_4screens_6_sdxl_1_wan_mix"
# dataset_name = "dim/render_nfs_4screens_5_sdxl_1_wan_mix"
dataset = load_dataset(
    dataset_name,
    cache_dir=f"dataset/{dataset_name.split('/')[-1]}",
)
# dataset["train"] = dataset["train"].shuffle(seed=2025)
dataset = dataset["train"]

In [5]:
from auto_remaster.sandbox.flux2_tiny_autoencoder import Flux2TinyAutoEncoder
import torch
from torchvision import transforms

resolution = 512
train_transforms = transforms.Compose(
    [
        transforms.Resize(
            resolution,
            interpolation=transforms.InterpolationMode.LANCZOS,
        ),
        transforms.CenterCrop(resolution),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5, 0.5, 0.5),
            (0.5, 0.5, 0.5),
        ),
    ]
)

weight_dtype = torch.bfloat16
device = "cuda"
vae_val = Flux2TinyAutoEncoder.from_pretrained(
    "fal/FLUX.2-Tiny-AutoEncoder",
    torch_dtype=weight_dtype,
).to(device)
vae_val.requires_grad_(False)
vae_val.eval()
None

c:\programming\auto_remaster\venv\Lib\site-packages\huggingface_hub\utils\_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
The config attributes {'auto_map': {'AutoModel': 'flux2_tiny_autoencoder.Flux2TinyAutoEncoder'}} were passed to Flux2TinyAutoEncoder, but are not expected and will be ignored. Please verify your config.json configuration file.


In [10]:
pos = 0
item = dataset[pos]
source_image_name = "input_image"
target_image_name = "edited_image"
# Подготовка исходных изображений для визуализации и метрик
orig_source_pil = item[source_image_name].convert("RGB")
target_pil = item[target_image_name].convert("RGB")
img_tensor = train_transforms(orig_source_pil).to(weight_dtype).to("cuda").unsqueeze(0)

latents = vae_val.encode(img_tensor, return_dict=False)
recon = vae_val.decode(latents, return_dict=False)
recon = recon.squeeze(0).clamp(-1, 1) / 2.0 + 0.5

In [12]:
import time

amount = 100
start = time.time()

for _ in range(amount):
    latents = vae_val.encode(img_tensor, return_dict=False)
    recon = vae_val.decode(latents, return_dict=False)
    recon = recon.squeeze(0).clamp(-1, 1) / 2.0 + 0.5

total = time.time() - start
print(f"total {total}s, {total/amount}s per img {1/(total/amount)}fps")

total 3.8698999881744385s, 0.038698999881744384s per img 25.840461072786884fps
